<a href="https://colab.research.google.com/github/AideeCCL/MineriaDatos/blob/main/RedesNeuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear csv

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# Configurar las credenciales
client_id = 'e41e2e2799f849dcb43211946125aa51'
client_secret = 'cf391168d4c5463688fed7000304976a'

# Autenticar
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Función para obtener características de una pista
def get_track_features(track_id):
    try:
        features = sp.audio_features(track_id)[0]
        return features
    except:
        return None

# Función para obtener información de una pista
def get_track_info(track_id):
    try:
        track = sp.track(track_id)
        return track
    except:
        return None

# Lista para almacenar los datos
data = []

# Obtener varias listas de reproducción populares
playlists = sp.category_playlists(category_id='pop')['playlists']['items']

for playlist in playlists:
    playlist_tracks = sp.playlist_tracks(playlist['id'])
    for item in playlist_tracks['items']:
        if item['track']:
            track_id = item['track']['id']

            # Obtener características de audio
            features = get_track_features(track_id)

            if features:
                # Obtener información de la pista
                track_info = get_track_info(track_id)

                if track_info:
                    # Crear un diccionario con la información relevante
                    track_data = {
                        'track_id': track_id,
                        'name': track_info['name'],
                        'artist': track_info['artists'][0]['name'],
                        'popularity': track_info['popularity'],
                        'danceability': features['danceability'],
                        'energy': features['energy'],
                        'key': features['key'],
                        'loudness': features['loudness'],
                        'mode': features['mode'],
                        'speechiness': features['speechiness'],
                        'acousticness': features['acousticness'],
                        'instrumentalness': features['instrumentalness'],
                        'liveness': features['liveness'],
                        'valence': features['valence'],
                        'tempo': features['tempo'],
                        'duration_ms': features['duration_ms']
                    }

                    data.append(track_data)

        # Respetar los límites de la API
        time.sleep(0.2)

# Crear un DataFrame y guardarlo como CSV
df = pd.DataFrame(data)
df.to_csv('spotify_dataset.csv', index=False)

print(f"Dataset creado con {len(df)} canciones.")

Dataset creado con 1533 canciones.


Recomendaciones de música en Spotify utilizando Python

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Cargar el dataset
df = pd.read_csv('spotify_dataset.csv')

# Seleccionar características para el modelo
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

X = df[features]
y = df['popularity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Evaluar el modelo
loss = model.evaluate(X_test_scaled, y_test)
print(f"Test loss: {loss}")

# Función para recomendar canciones
def recommend_songs(input_song, num_recommendations=5):
    input_features = df[df['name'] == input_song][features]
    if input_features.empty:
        print(f"La canción '{input_song}' no se encuentra en el dataset.")
        return None

    input_features_scaled = scaler.transform(input_features)

    all_songs = df[df['name'] != input_song]
    all_features = all_songs[features]
    all_features_scaled = scaler.transform(all_features)

    predictions = model.predict(all_features_scaled)
    all_songs['predicted_popularity'] = predictions

    recommended_songs = all_songs.sort_values('predicted_popularity', ascending=False).head(num_recommendations)
    return recommended_songs[['name', 'artist', 'predicted_popularity']]

# Ejemplo de uso
input_song = "Die With A Smile"  # Asegúrate de que esta canción esté en tu dataset
recommendations = recommend_songs(input_song)
if recommendations is not None:
    print(f"Recomendaciones para '{input_song}':")
    print(recommendations)

# Función para obtener las canciones disponibles en el dataset
def get_available_songs():
    return df['name'].unique()

# Imprimir algunas canciones disponibles
print("\nAlgunas canciones disponibles en el dataset:")
print(get_available_songs()[:10])  # Muestra las primeras 10 canciones

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 4431.2383 - val_loss: 4191.2622
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4354.6333 - val_loss: 3686.5964
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3579.5671 - val_loss: 2191.5942
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1814.6965 - val_loss: 650.5968
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 672.6087 - val_loss: 488.7971
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 635.9092 - val_loss: 449.9952
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 570.2070 - val_loss: 431.0030
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 572.8666 - val_loss: 422.7350
Epoch 9/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 520.2707 - val_loss: 413.0633
Epoch 10/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 517.1022 - val_loss: 408.3780
Epoch 11/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 540.6953 - val_loss: 403.1138
Epoch 12/100
31/31 ━━━━━

<ipython-input-9-a4a22920e51d>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_songs['predicted_popularity'] = predictions


Modificación con el algoritmo de Kmeans

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Cargar el dataset
df = pd.read_csv('spotify_dataset.csv')

# Seleccionar características para el modelo
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

X = df[features]
y = df['popularity']

# Normalizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar K-means
n_clusters = 10  # Puedes ajustar este número
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Crear el modelo de red neuronal
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(features),)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

# Función para recomendar canciones
def recommend_songs(input_song, num_recommendations=5):
    input_features = df[df['name'] == input_song][features]
    if input_features.empty:
        print(f"La canción '{input_song}' no se encuentra en el dataset.")
        return None

    input_features_scaled = scaler.transform(input_features)
    input_cluster = kmeans.predict(input_features_scaled)[0]

    # Filtrar canciones del mismo cluster
    cluster_songs = df[df['cluster'] == input_cluster]
    cluster_features = scaler.transform(cluster_songs[features])

    predictions = model.predict(cluster_features)
    cluster_songs['predicted_popularity'] = predictions

    recommended_songs = cluster_songs[cluster_songs['name'] != input_song].sort_values('predicted_popularity', ascending=False).head(num_recommendations)
    return recommended_songs[['name', 'artist', 'predicted_popularity']]

# Ejemplo de uso
input_song = "Shape of You"  # Asegúrate de que esta canción esté en tu dataset
recommendations = recommend_songs(input_song)
if recommendations is not None:
    print(f"Recomendaciones para '{input_song}':")
    print(recommendations)

# Función para obtener las canciones disponibles en el dataset
def get_available_songs():
    return df['name'].unique()

# Imprimir algunas canciones disponibles
print("\nAlgunas canciones disponibles en el dataset:")
print(get_available_songs()[:10])  # Muestra las primeras 10 canciones

# Función para analizar los clusters
def analyze_clusters():
    cluster_stats = df.groupby('cluster').agg({
        'popularity': 'mean',
        'name': 'count'
    }).rename(columns={'name': 'song_count'})

    print("Estadísticas de los clusters:")
    print(cluster_stats)

    # Encontrar las canciones más populares en cada cluster
    for cluster in range(n_clusters):
        top_songs = df[df['cluster'] == cluster].nlargest(3, 'popularity')
        print(f"\nCanciones más populares en el cluster {cluster}:")
        print(top_songs[['name', 'artist', 'popularity']])

# Analizar los clusters
analyze_clusters()

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 4441.1118 - val_loss: 4170.9053
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4180.7632 - val_loss: 3649.1365
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3472.8748 - val_loss: 2190.5210
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1794.8240 - val_loss: 628.5151
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 735.6337 - val_loss: 495.6064
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 582.9518 - val_loss: 452.9461
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 531.1510 - val_loss: 437.4713
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 614.3256 - val_loss: 426.2715
Epoch 9/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 576.5728 - val_loss: 418.7040
Epoch 10/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 540.4146 - val_loss: 416.2657
Epoch 11/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 538.3864 - val_loss: 410.2125
Epoch 12/100
31/31 ━━━━━